<a href="https://colab.research.google.com/github/cuong3004/internship/blob/duccuong/Demo_Multiple_Label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
!pip install -U gdown transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.5 MB 14.8 MB/s 
     |████████████████████████████████| 6.8 MB 27.2 MB/s 
     |████████████████████████████████| 895 kB 32.9 MB/s 
     |████████████████████████████████| 596 kB 36.3 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
  Created wheel for gdown: filename=gdown-4.3.1-py3-none-any.whl size=14493 sha256=59a490ad920425da0715a11303f3f43bc6a2a9af33fc7dfa210bc87d99f90ed1
  Stored in directory: /root/.cache/pip/wheels/39/13/56/88209f07bace2c1af0614ee3326de4a00aad74afb0f4be921d
Successfully built gdown
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.1
    Uninstalling gdown-4.2.1:
      Successfully uninstalled gdown-4.2.1


In [ ]:
!gdown --id "1-GQ9GPpPL5tNEG5I7ooGbtn2CBqX50dS"
!gdown --id "1-4aCuw_XZ1iOCL824EKE1KsKsGL7EwXG"

Downloading...
From: https://drive.google.com/uc?id=1-GQ9GPpPL5tNEG5I7ooGbtn2CBqX50dS
To: /content/Question_Classification_Dataset.csv
100% 409k/409k [00:00<00:00, 15.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-4aCuw_XZ1iOCL824EKE1KsKsGL7EwXG
To: /content/traced_bert.pt
100% 439M/439M [00:08<00:00, 53.7MB/s]


In [ ]:
from transformers import AutoTokenizer
import torch 
import pandas as pd
from pprint import pprint

df = pd.read_csv("/content/Question_Classification_Dataset.csv", index_col=False)

label1_list = sorted(list(set(df.iloc[:,-3])))
label2_list = sorted(list(set(df.iloc[:,-1])))
label1_dir = {k:v for v, k in enumerate(label1_list)}
label2_dir = {k:v for v, k in enumerate(label2_list)}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = torch.jit.load("traced_bert.pt")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def preprocessing_input(text):
    encoded_dict = tokenizer.encode_plus(
                text,                      
                add_special_tokens = True,
                max_length = 20,           
                pad_to_max_length = True,
                return_attention_mask = True,   
                return_tensors = 'pt',     
            )
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']

    return input_id, attention_mask

def predict(text):

    input_id, attention_mask = preprocessing_input(text)
    with torch.no_grad():
        out1, out2 = model(input_id, attention_mask)
    out1_confident = torch.softmax(out1[0], -1)
    out1_argmax = torch.argmax(out1_confident, -1).item()
    out1_confident = out1_confident[out1_argmax].item()
    out2_confident = torch.softmax(out2[0], -1)
    out2_argmax = torch.argmax(out2_confident, -1).item()
    out2_confident = out2_confident[out2_argmax].item()

    confident_label = out2_confident*out1_confident
    return {
        "text": text,
        "category": {
            "name": label1_list[out1_argmax],
            "confident": out1_confident,
        },
        "subcategory": {
            "name": label2_list[out2_argmax],
            "confident": out2_confident,
        },
        "confident_label": confident_label,
    }



## Demo

In [ ]:
#@title Write text
my_text = "The most conductive metal" #@param {type:"string"}


pprint(predict(my_text))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'category': {'confident': 0.9861424565315247, 'name': 'ENTITY'},
 'confident_label': 0.7098780601649146,
 'subcategory': {'confident': 0.7198534607887268, 'name': 'substance'},
 'text': 'The most conductive metal'}
